This is the FFI group. Since we still haven’t received the data from the client, we are using a generic plain text to SQL dataset provided to us by Professor Chatterjee. This data is a large crowd-sourced dataset that contains generic queries on some generic relational database called “table”. There are three csv files for training, testing and validation which are already cleaned and pre-processed. Still, we are including a preprocessor for the plain text that removes punctuation and removes uppercase letters. Here is the citation for the work:



@article{zhongSeq2SQL2017, 
author = {Victor Zhong and
Caiming Xiong and
Richard Socher},
title = {Seq2SQL: Generating Structured Queries from Natural Language using
Reinforcement Learning},
journal = {CoRR},
volume = {abs/1709.00103},
year = {2017}
}


In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 4.2 MB/s eta 0:00:00


In [ ]:
# dependencies goes here
from google.colab import files
import io
import pandas as pd
import regex as re
import gensim.downloader
import spacy
from datasets import load_dataset
from tqdm.auto import tqdm


In [ ]:
# code for exploration
dataset = load_dataset('wikisql')

Generating test split:   0%|          | 0/15878 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8421 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/56355 [00:00<?, ? examples/s]

Dataset wikisql downloaded and prepared to /root/.cache/huggingface/datasets/wikisql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# dataset['train'][0]
question_set = []
for row in tqdm(dataset['train']):
  data = {
      'question': row['question']
          }
  question_set.append(data)
  # didn't change to lower case as capital letters might imply names or location
question_df = pd.DataFrame(question_set)

  0%|          | 0/56355 [00:00<?, ?it/s]

In [ ]:
# exploring sapcy
# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")
print('-'*10)
nlp_attrs = vars(nlp)
print(',\n'.join("%s: %s" % item for item in nlp_attrs.items()))
print('-'*10)

# text = "text"
# doc = nlp(text)

# Analyze syntax
# print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
# print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
# text = question_df.iloc[1][0]
# doc = nlp(text)
# entities = []
# entities_label = []
# label_explanation = {}
# for entity in doc.ents:
#   entities.append(entity.text)
#   entities_label.append(entity.label_)
#   label_explanation[entity.label_] = spacy.explain(entity.label_)
# print(entities, entities_label, label_explanation)


----------
_config: {'paths': {'train': None, 'dev': None, 'vectors': None, 'init_tok2vec': None}, 'system': {'gpu_allocator': None, 'seed': 0}, 'nlp': {'lang': 'en', 'pipeline': ['tok2vec', 'tagger', 'parser', 'senter', 'attribute_ruler', 'lemmatizer', 'ner'], 'disabled': ['senter'], 'before_creation': None, 'after_creation': None, 'after_pipeline_creation': None, 'batch_size': 256, 'tokenizer': {'@tokenizers': 'spacy.Tokenizer.v1'}}, 'components': {'attribute_ruler': {'factory': 'attribute_ruler', 'scorer': {'@scorers': 'spacy.attribute_ruler_scorer.v1'}, 'validate': False}, 'lemmatizer': {'factory': 'lemmatizer', 'mode': 'rule', 'model': None, 'overwrite': False, 'scorer': {'@scorers': 'spacy.lemmatizer_scorer.v1'}}, 'ner': {'factory': 'ner', 'incorrect_spans_key': None, 'moves': None, 'scorer': {'@scorers': 'spacy.ner_scorer.v1'}, 'update_with_oracle_cut_size': 100, 'model': {'@architectures': 'spacy.TransitionBasedParser.v2', 'state_type': 'ner', 'extra_state_tokens': False, 'hidd

In [7]:
nlp = spacy.load("en_core_web_sm")
def extract(text: str):
  # print(text)
  entities = []
  entities_label = []
  label_explanation = {}
  doc = nlp(text)
  for entity in doc.ents:
    entities.append(entity.text)
    entities_label.append(entity.label_)
    label_explanation[entity.label_] = spacy.explain(entity.label_)
  return [entities, entities_label, label_explanation]
tqdm.pandas(desc='Extracting')
extracted = question_df['question'].progress_apply(extract)
# question_df['question'].transform(extract)
question_df[['entities','entities_label','label_explanation']] = pd.DataFrame(extracted.tolist(), index= question_df.index)



Extracting:   0%|          | 0/56355 [00:00<?, ?it/s]

In [8]:
question_df.to_csv('/content/wikisql_question_extracted_info.csv', index=False)
files.download('/content/wikisql_question_extracted_info.csv')
question_df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,question,entities,entities_label,label_explanation
0,Tell me what the notes are for South Australia,[South Australia],[GPE],"{'GPE': 'Countries, cities, states'}"
1,What is the current series where the new serie...,[June 2011],[DATE],{'DATE': 'Absolute or relative dates or periods'}
2,What is the format for South Australia?,[South Australia],[GPE],"{'GPE': 'Countries, cities, states'}"
3,Name the background colour for the Australian ...,[the Australian Capital Territory],[ORG],"{'ORG': 'Companies, agencies, institutions, et..."
4,how many times is the fuel propulsion is cng?,[],[],{}
...,...,...,...,...
56350,What time was the match played with a score of...,[3-2],[CARDINAL],{'CARDINAL': 'Numerals that do not fall under ...
56351,On which ground did the team play Aston Villa?,[Aston Villa],[PERSON],"{'PERSON': 'People, including fictional'}"
56352,What kind of competition was it at San Siro at...,"[San Siro, 18:30 GMT]","[GPE, TIME]","{'GPE': 'Countries, cities, states', 'TIME': '..."
56353,What is the total number of decile for the red...,[],[],{}


In [10]:
nlp = spacy.load("en_core_web_sm")
question_df_2 = question_df.copy()
def part_of_speech(text:str):
  '''
  refers to this documentation: https://spacy.io/usage/linguistic-features#pos-tagging
  '''
  # text = []
  entities_pos = [] # the postion of the token in part-of-speech
  entities_tag = []
  entities_shape =  []
  entities_dep = []

  label_explanation = {}
  # special_case = [{ORTH: "Virat"}, {ORTH: "Kholi"}]        # We can add our special cases here to make the explaination more reasonable 
  # nlp.tokenizer.add_special_case("ViratKholi", special_case)
  doc = nlp(text)
  for entity in doc:
    entities_pos.append(entity.pos_)
    entities_tag.append(entity.tag_)
    entities_shape.append(entity.shape_)
    entities_dep.append(entity.dep_)
  return [entities_pos, entities_tag, entities_shape,entities_dep]


tqdm.pandas(desc='Extracting')
extracted = question_df_2['question'].progress_apply(part_of_speech)
# question_df['question'].transform(extract)
question_df_2[['entities_pos','entities_tag','entities_shape','entities_dep']] = pd.DataFrame(extracted.tolist(), index= question_df.index)

Extracting:   0%|          | 0/56355 [00:00<?, ?it/s]

In [11]:
question_df.head

<bound method NDFrame.head of                                                 question  \
0        Tell me what the notes are for South Australia    
1      What is the current series where the new serie...   
2                What is the format for South Australia?   
3      Name the background colour for the Australian ...   
4          how many times is the fuel propulsion is cng?   
...                                                  ...   
56350  What time was the match played with a score of...   
56351     On which ground did the team play Aston Villa?   
56352  What kind of competition was it at San Siro at...   
56353  What is the total number of decile for the red...   
56354        Which report includes a Circuit of Tripoli?   

                                 entities entities_label  \
0                       [South Australia]          [GPE]   
1                             [June 2011]         [DATE]   
2                       [South Australia]          [GPE]   
3      [t

In [12]:
question_df_2.to_csv('/content/wikisql_question_extracted_partOfSpeech.csv', index=False)
files.download('/content/wikisql_question_extracted_partOfSpeech.csv')
question_df_2

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,question,entities,entities_label,label_explanation,entities_pos,entities_tag,entities_shape,entities_dep
0,Tell me what the notes are for South Australia,[South Australia],[GPE],"{'GPE': 'Countries, cities, states'}","[VERB, PRON, PRON, DET, NOUN, AUX, ADP, PROPN,...","[VB, PRP, WP, DT, NNS, VBP, IN, NNP, NNP]","[Xxxx, xx, xxxx, xxx, xxxx, xxx, xxx, Xxxxx, X...","[ROOT, dobj, pobj, det, nsubj, ccomp, prep, co..."
1,What is the current series where the new serie...,[June 2011],[DATE],{'DATE': 'Absolute or relative dates or periods'},"[PRON, AUX, DET, ADJ, NOUN, SCONJ, DET, ADJ, N...","[WP, VBZ, DT, JJ, NN, WRB, DT, JJ, NN, VBD, IN...","[Xxxx, xx, xxx, xxxx, xxxx, xxxx, xxx, xxx, xx...","[attr, ROOT, det, amod, nsubj, advmod, det, am..."
2,What is the format for South Australia?,[South Australia],[GPE],"{'GPE': 'Countries, cities, states'}","[PRON, AUX, DET, NOUN, ADP, PROPN, PROPN, PUNCT]","[WP, VBZ, DT, NN, IN, NNP, NNP, .]","[Xxxx, xx, xxx, xxxx, xxx, Xxxxx, Xxxxx, ?]","[attr, ROOT, det, nsubj, prep, compound, pobj,..."
3,Name the background colour for the Australian ...,[the Australian Capital Territory],[ORG],"{'ORG': 'Companies, agencies, institutions, et...","[VERB, DET, NOUN, NOUN, ADP, DET, PROPN, PROPN...","[VB, DT, NN, NN, IN, DT, NNP, NNP, NNP]","[Xxxx, xxx, xxxx, xxxx, xxx, xxx, Xxxxx, Xxxxx...","[ROOT, det, compound, dobj, prep, det, compoun..."
4,how many times is the fuel propulsion is cng?,[],[],{},"[SCONJ, ADJ, NOUN, AUX, DET, NOUN, NOUN, AUX, ...","[WRB, JJ, NNS, VBZ, DT, NN, NN, VBZ, NN, .]","[xxx, xxxx, xxxx, xx, xxx, xxxx, xxxx, xx, xxx...","[advmod, amod, attr, ROOT, det, compound, nsub..."
...,...,...,...,...,...,...,...,...
56350,What time was the match played with a score of...,[3-2],[CARDINAL],{'CARDINAL': 'Numerals that do not fall under ...,"[DET, NOUN, AUX, DET, NOUN, VERB, ADP, DET, NO...","[WDT, NN, VBD, DT, NN, VBN, IN, DT, NN, IN, CD...","[Xxxx, xxxx, xxx, xxx, xxxx, xxxx, xxxx, x, xx...","[det, attr, ROOT, det, attr, acl, prep, det, p..."
56351,On which ground did the team play Aston Villa?,[Aston Villa],[PERSON],"{'PERSON': 'People, including fictional'}","[ADP, DET, NOUN, AUX, DET, NOUN, VERB, PROPN, ...","[IN, WDT, NN, VBD, DT, NN, VB, NNP, NNP, .]","[Xx, xxxx, xxxx, xxx, xxx, xxxx, xxxx, Xxxxx, ...","[prep, det, pcomp, aux, det, nsubj, ROOT, comp..."
56352,What kind of competition was it at San Siro at...,"[San Siro, 18:30 GMT]","[GPE, TIME]","{'GPE': 'Countries, cities, states', 'TIME': '...","[DET, NOUN, ADP, NOUN, AUX, PRON, ADP, PROPN, ...","[WDT, NN, IN, NN, VBD, PRP, IN, NNP, NNP, IN, ...","[Xxxx, xxxx, xx, xxxx, xxx, xx, xx, Xxx, Xxxx,...","[det, attr, prep, pobj, ROOT, nsubj, prep, com..."
56353,What is the total number of decile for the red...,[],[],{},"[PRON, AUX, DET, ADJ, NOUN, ADP, NOUN, ADP, DE...","[WP, VBZ, DT, JJ, NN, IN, NN, IN, DT, NN, NN, ...","[Xxxx, xx, xxx, xxxx, xxxx, xx, xxxx, xxx, xxx...","[attr, ROOT, det, amod, nsubj, prep, pobj, pre..."


In [ ]:
# tests 